# Solving 3D Poisson Equation with RBF-FD Method
## Problem Setup
**PDE**: $-\nabla^2 u = f$ in unit cube $[0,1]^3$  
**Exact Solution**: $u(x,y,z) = \sin(\pi x)\sin(\pi y)\sin(\pi z)$  
**Source Term**: $f = 3\pi^2 u$  
**Boundary**: Dirichlet conditions from exact solution

## 1. Import Required Libraries

In [ ]:
import numpy as np
from scipy.spatial import KDTree
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 2. Parameters Configuration
Key parameters controlling the solution:
- `n`: Number of nodes along each axis (total nodes = $n^3$)
- `stencil_size`: Number of neighbors for local approximation
- `poly_order`: Polynomial augmentation order (1=linear, 2=quadratic)
- `regularization`: Small value to stabilize matrix inversion

In [ ]:
# Computational parameters
n = 10                # Nodes per axis
stencil_size = 30     # Neighbors per stencil
poly_order = 2        # 1=linear, 2=quadratic
regularization = 1e-8 # Matrix stabilization

# RBF parameters
phi_order = 3         # Polyharmonic spline order (r^3)
phi_factor = 12       # Laplacian factor for r^3: ∇²(r^3) = 12r

## 3. Node Generation
Creates structured grid nodes and identifies boundary nodes:

In [ ]:
def generate_nodes(n):
    """Create structured grid in unit cube"""
    x = np.linspace(0, 1, n)
    X, Y, Z = np.meshgrid(x, x, x)
    return np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

def identify_boundary(nodes, tol=1e-5):
    """Flag nodes on cube surfaces"""
    return np.any((nodes <= tol) | (nodes >= 1 - tol), axis=1)

# Generate nodes and boundary flags
nodes = generate_nodes(n)
bndry = identify_boundary(nodes)
interior = ~bndry
N = nodes.shape[0]

## 4. Polynomial Setup
Configures polynomial augmentation based on selected order:

| Order | Terms Included | Count |
|-------|----------------|-------|
| 1     | 1, x, y, z     | 4     |
| 2     | +x²,y²,z²,xy,xz,yz | 10 |
| 3     | +x³,y³,z³,x²y,... | 20 |

In [ ]:
# Polynomial configuration
if poly_order == 1:
    m = 4  # Linear terms
elif poly_order == 2:
    m = 10 # Quadratic terms
else:
    m = 20 # Cubic terms

# Initialize KDTree for neighbor search
tree = KDTree(nodes)

## 5. Laplacian Matrix Assembly
Constructs sparse differentiation matrix using RBF-FD:

In [ ]:
L = lil_matrix((N, N))

for i in np.where(interior)[0]:
    # Get stencil neighbors
    _, indices = tree.query(nodes[i], k=stencil_size)
    stencil_nodes = nodes[indices]
    n_stencil = len(indices)
    
    # Build RBF matrix (phi(r) = r^3)
    A = np.zeros((n_stencil, n_stencil))
    for j in range(n_stencil):
        for k in range(j, n_stencil):
            r = np.linalg.norm(stencil_nodes[j] - stencil_nodes[k])
            A[j,k] = r**phi_order
            A[k,j] = A[j,k]  # Symmetric
    
    # Add regularization
    A += regularization * np.eye(n_stencil)
    
    # Build polynomial matrix
    P = np.ones((n_stencil, m))
    P[:, 1:4] = stencil_nodes  # x, y, z
    if poly_order >= 2:
        P[:, 4:7] = stencil_nodes**2  # x², y², z²
        # Cross terms
        cross_terms = np.column_stack([
            stencil_nodes[:,0]*stencil_nodes[:,1],  # xy
            stencil_nodes[:,0]*stencil_nodes[:,2],  # xz
            stencil_nodes[:,1]*stencil_nodes[:,2]   # yz
        ])
        P[:, 7:10] = cross_terms
    
    # RHS for Laplacian approximation
    center = nodes[i]
    b_rbf = np.array([phi_factor * np.linalg.norm(node - center) 
                     for node in stencil_nodes])
    
    # Polynomial consistency RHS
    b_poly = np.zeros(m)
    if poly_order >= 2:
        b_poly[4:7] = 2.0  # ∇²(x²)=2
    
    # Solve augmented system
    aug_A = np.block([[A, P], [P.T, np.zeros((m, m))]])
    aug_b = np.hstack([b_rbf, b_poly])
    weights = np.linalg.lstsq(aug_A, aug_b, rcond=1e-6)[0][:n_stencil]
    
    # Store weights
    L[i, indices] = weights

## 6. Boundary Conditions
Enforce Dirichlet conditions using exact solution:

In [ ]:
# Exact solution function
u_exact = lambda x: np.sin(np.pi*x[:,0]) * np.sin(np.pi*x[:,1]) * np.sin(np.pi*x[:,2])

# Apply BCs to matrix
L[bndry] = 0
L[bndry, bndry] = 1.0
L = L.tocsr()  # Convert to efficient format

## 7. Solve Linear System
Assemble RHS and solve using sparse solver:

In [ ]:
# Source term (note negative sign for -∇²u = f)
f = 3 * np.pi**2 * u_exact(nodes)
rhs = -f.copy()
rhs[bndry] = u_exact(nodes[bndry])

# Solve
u = spsolve(L, rhs)

## 8. Post-processing
Calculate errors and prepare visualization:

In [ ]:
# Error calculation
error = u - u_exact(nodes)
relative_error = np.linalg.norm(error) / np.linalg.norm(u_exact(nodes))
print(f"Relative L2 Error: {relative_error:.4e}")

# Reshape for 3D visualization
grid_shape = (n, n, n)
U = u.reshape(grid_shape)
U_exact = u_exact(nodes).reshape(grid_shape)
error_3d = np.abs(U - U_exact)

## 9. Visualization
3D surface plots of solutions and error:

In [ ]:
# Slice at z=0.5
iz = n//2
x = y = np.linspace(0, 1, n)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(18, 5))

# Numerical Solution
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, U[:, :, iz], cmap='viridis')
ax1.set_title('Numerical Solution')

# Exact Solution
ax2 = fig.add_subplot(132, projection='3d')
ax2.plot_surface(X, Y, U_exact[:, :, iz], cmap='viridis')
ax2.set_title('Exact Solution')

# Error
ax3 = fig.add_subplot(133, projection='3d')
ax3.plot_surface(X, Y, error_3d[:, :, iz], cmap='hot')
ax3.set_title('Absolute Error')

plt.tight_layout()
plt.show()